Weitere wichtige Graphalgorithmen
=================


Topologisches Sortieren
------------------------

Wir konzentrieren uns zunächst auf gerichtete Graphen mit einer bestimmten Eigenschaft:

**Directed acyclic graphs** (gerichtete azyklische Graphen), kurz **DAGs**

Ein gerichteter azyklischer Graph ist ein gerichteter Graph, in dem sich keine Zyklen finden lassen. Unsere Beispiele werden meist zusammenhängend sein, d.h., wenn wir jede Kante als ungerichtete Kante ansehen, dann erhalten wir einen ungerichteten Graphen, der zusammenhängend ist, also in dem jeder Knoten von jedem Knoten aus erreichbar ist.

Mit diesen DAGs lassen sich z.B. Reihenfolge-Constraints (also Zwänge bzw. Bedingungen, die man erfüllen muss) sehr gut darstellen.

Schauen wir uns ein einfaches Beispiel an, einen kleinen Ausschnitt aus dem Studienverlaufsplan der Wirtschaftsinformatiker, s. S.12ff in https://www.w-hs.de/fileadmin/public/user_upload/BPO_Wirtschaftsinformatik_2016.pdf:

<pre>
EPR ---> OPR 
LDS ---> ADS 
EBW ---> PMA  # MUSS-Bedingung laut Prüfungsordnung
GWI ---> PMA  # MUSS (= Hard-Constraint)
EPR ---> BI1 # MUSS
GWI ---> BI1 # MUSS
EBW ---> BI1 # MUSS
PMW ---> BI1 # MUSS
ADS ---> Projekt # MUSS
EPR ---> Projekt # MUSS
OPR ---> Projekt # MUSS (hier logisch redundant, aber EPR ---> OPR ist kein MUSS)
EPR ---> BI2 # MUSS
GWI ---> BI2 # MUSS
EBW ---> BI2 # MUSS
PMW ---> BI2 # MUSS
SWT ---> BI2 # MUSS
OPR ---> BI2 # MUSS
BI1 ---> Studienabschluss
BI2 ---> Studienabschluss
Projekt ---> Studienabschluss
</pre>


Nun stellen wir die folgende Frage:

_In welcher Reihenfolge kann ich die Prüfungen der Kurse ablegen, damit ich mein Ziel "Studienabschluss" erreiche?_

Die Antwort zu der Frage gibt uns eine "Linearisierung" der Kurse, also eine lineare Abfolge der Kurse, die so gewählt wird, dass keiner der Reihenfolge-Constraints verletzt wird, also für jeden Constraint X ---> Y gilt, dass X in der linearisierten Reihenfolge vor Y auftaucht.

Einige Beobachtungen:

(1) Weil wir einen DAG haben (haben wir, kontrollieren Sie es!), kann man sicher eine solche linerare Ordnung finden.

(2) Es gibt eine ganze Reihe von denkbaren Startpunkten, nämlich alle "Knoten", also Fächer, die keine eingehende Kante aufweisen.

(3) Wenn wir zuerst alle Konten mit Eingangsgrad 0 betrachten und unter diesen irgendeine Reihenfolge wählen, dann können wir damit nichts falsch machen.

(4) Es gibt immer mindestens einen Knoten mit Eingangsgrad 0 (sonst hätten wir mindestens einen Zyklus).

(5) Wenn wir alle Knoten mit Eingangsgrad 0 "versorgt" haben (also eine beliebige lineare Ordnung unter diesen gewählt haben), dann können wir diese und alle von Ihnen ausgehenden Kanten entfernen und erhalten wieder einen DAG (klar, das ENTFERNEN von Kanten kann nicht pläötzlich zu einem Kreis führen).

(6) Für diesen DAG gehen wir dann so vor, wie in (2)-(5) ... bis keine Knoten mehr übrig sind. Dann haben wir eine lineare Ordnung gefunden!

Dies ist ein möglicher Algorithmus, um eine sogenannte **Topologische Sortierung** für einen DAG zu finden. Die lineare Ordnung, die wir am Ende abliefern, ist das Ergebnis der topologischen Sortierung, manchmal wird aber auch direkt das Ergebnis so genannt.

Machen Sie das für den obigen Graphen!


In [17]:
# Lösen wir das mit einem Algorithmus

# Zunächst brauchen wir eine Repräsentation des Graphen, eine einfache Möglichkeit ergibt sich 
# über Adjazenzlisten, die in einem Dictionary (z.B. HashMap in JAVA) zusammengefasst werden
# Damit sind auch gleichzeitig die Knoten im Graph bestimmt

graph = { 
    'EPR': ['OPR','BI1', 'Projekt'],
    'LDS': ['ADS'],
    'EBW': ['PMA', 'BI1', 'BI2'],
    'GWI': ['PMA','BI1','BI2'],
    'PMW': ['BI1','BI2'],
    'ADS': ['Projekt'],           
    'OPR': ['Projekt','BI2'],
    'SWT': ['BI2'],
    'BI1': ['Studienabschluss'],
    'BI2': ['Studienabschluss'],
    'Projekt': ['Studienabschluss'],
    # Jetzt gibt es einige Knoten, die nicht links im Dictionary auftauchen (die also nicht
    # Voraussetzung für andere Fächer sind).
    # Diese könnten wir "zu Fuss" erfassen oder automatisiert finden. Wir erfassen Sie "zu Fuss":
    'Studienabschluss': [],
    'PMA': []
}

# Jetzt suchen wir eine lineare Ordnung zu einem gegebenen DAG, dies nennt sich:
# Topologisches Sortieren

# Wir wollen das wie oben angegeben machen.

# Hilfsfunktion - geht eleganter in Python und ist so zudem schrecklich ineffizient,
# dafür aber leicht zu verstehen
def hat_eingehende_kante(g,k):  # g ist ein Graph, k eine Kante
    knoten = graph.keys()
    for kn in knoten:
        if k in graph[kn]:
            return True
    return False

def topological_sort(graph):
    
    if len(graph) == 0: # Graph ist leer
        return []

    # Finde Knoten mit Eingangsgrad 0:
    #  Alle Knoten, die nicht in den Adjazenzlisten auftauchen!
    knoten = graph.keys() # Knoten, die noch im Graph sind
    # Suche alle heraus, die keine eingehende Kante haben
    eingangsgrad_0 = [k for k in knoten if not hat_eingehende_kante(graph,k)]
    print("Nodes to remove next: ", eingangsgrad_0) # Ausgabe zur Kontrolle
    
    # Entferne diese Knoten und Kanten aus dem Graph
    for k in eingangsgrad_0:
        del graph[k]
    
    return eingangsgrad_0 + topological_sort(graph) # das + vereinigt zwei Listen 
    
    
print("\nLinear order: ", topological_sort(graph))

Nodes to remove next:  ['EPR', 'LDS', 'EBW', 'GWI', 'PMW', 'SWT']
Nodes to remove next:  ['ADS', 'OPR', 'BI1', 'PMA']
Nodes to remove next:  ['BI2', 'Projekt']
Nodes to remove next:  ['Studienabschluss']

Linear order:  ['EPR', 'LDS', 'EBW', 'GWI', 'PMW', 'SWT', 'ADS', 'OPR', 'BI1', 'PMA', 'BI2', 'Projekt', 'Studienabschluss']


Dies ist eine einfache Vorgehensweise. Damit das funktioniert, setzen wir voraus, dass es immer Knoten mit dem Eingangsgrad 0 gibt (oder, dass es gar keine Knoten mehr gibt). Wenn das nicht erfüllt wäre, dann würden wir in einer Endlosschleife landen.

Wir könnten oben leicht eine Überprüfung einbauen, indem wir prüfen, ob die "Menge" (die wir als Liste darstellen) eingangsgrad_0 eine Länge von 0 hat. Damit würden wir einige Problemfälle erkennen (vielleicht sogar alle?), aber das ersparen wir uns hier (bauen Sie es selbst zur Übung ein und füttern Sie den Algo dann mit einem Graph mit einem Zyklus!).

Was wir noch erläutern sollten, ist die _list comprehension_, die wir oben verwendet haben:

<code>
    eingangsgrad_0 = [k for k in knoten if not hat_eingehende_kante(graph,k)]
</code>

Was bedeutet das? Das ist einfach eine sehr kompakte Schreibweise für eine Schleife mit Filterfunktion, die eine Liste erzeugt und direkt zuweist, also für das folgende Konstrukt:

<code>
    eingangsgrad_0 = []
    for k in knoten:
        if not hat_eingehende_kante(graph,k):
            eingangsgrad_0.append(k)
</code>

Da sieht die _list_comprehension_ doch wesentlich eleganter und mindestens genauso verständlich aus, oder?

Übrigens: für die Gruppen von Knoten, die oben entfernt werden, z.B. die Gruppe  

<pre>['ADS', 'OPR', 'BI1', 'PMA']</pre>

könnte man *jede Permutation* als Linearisierung wählen, also eine beliebige der 4\*3\*2\*1 = 24 möglichen Permutationen. Das gilt für jede Gruppe. Wenn man die Anzahl der Permutationen für die einzelnen Gruppen miteinander mutliziert, hat man eine Abschätzung der Anzahl korrekter linearer Ordnungen. Die kann dann sogar noch größer sein, wenn bestimmte Elemente in der nächsten Gruppe nicht von allen Elementen aus der vorigen Gruppe abhängig sind (z.B. ist PMA nicht von LDS abhängig, also könnte LDS aus NACH PMA geprüft werden), das kann auch über mehrere Ebenen hinweg auftreten usw. Aber so genau müssen wir das hier nicht betrachten, Hauptsache, wir wissen nun, wie wir eine topologische Sortierung finden können! 

(Für Spezialisten: wenn wir jede denkbare topologische Sortierung erzeugen können wollten, was müssten wir dann tun? Ja, genau, wir müssten mit jedem Knoten aus der Gruppe der Knoten mit Eingangsgrad 0 eine Linearisierung beginnen (bzw. fortführen) UND dann nur diesen einen Knoten und seine ausgehenden Kanten löschen usw. Dann würde jede mögliche Linearisierung entstehen).

Denkbare Aufgaben
------------------

Gegeben: Gerichteter Graph durch die Knoten und Kantenmengen. 

_Frage: Kann man diesen Graph topologisch sortieren? Wenn ja, dann geben sie eine topologische Sortierung als Sequenz der Knoten an._

Man muss also zuerst schauen, ob der Graph einen Zyklus aufweist und dann, falls nicht, eine lineare Ordnung finden und diese angeben, wie oben im Beispiel geschehen.

